In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
import psycopg

In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": dst_host, 
    "port": dst_port,
    "dbname": dst_db,
    "user": dst_username,
    "password": dst_password,
}

assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

In [9]:
# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [10]:
df

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,11532,0489-WMEMG,2018-03-01,NaT,One year,Yes,Electronic check,49.45,1119.35,DSL,...,No,Yes,No,No,Female,0,No,Yes,No,0
1,11534,7435-ZNUYY,2019-08-01,NaT,One year,No,Mailed check,20.60,116.60,None,...,None,None,None,None,Male,0,No,No,No,0
2,11536,1354-YZFNB,2019-10-01,NaT,Two year,No,Credit card (automatic),19.55,68.80,None,...,None,None,None,None,Male,0,Yes,Yes,No,0
3,11539,6956-SMUCM,2019-07-01,2019-10-01,Month-to-month,Yes,Credit card (automatic),99.00,287.40,Fiber optic,...,Yes,No,Yes,Yes,Female,0,No,No,No,1
4,11541,2985-FMWYF,2018-03-01,NaT,Month-to-month,Yes,Electronic check,93.50,2341.55,Fiber optic,...,No,No,Yes,Yes,Female,0,No,No,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,11523,0885-HMGPY,2014-12-01,NaT,Two year,No,Bank transfer (automatic),69.40,4237.50,DSL,...,Yes,Yes,No,No,Male,0,No,No,Yes,0
7039,11525,5003-OKNNK,2018-11-01,NaT,One year,No,Credit card (automatic),20.35,335.95,None,...,None,None,None,None,Female,0,Yes,Yes,No,0
7040,11527,4195-SMMNX,2019-10-01,2019-12-01,Month-to-month,No,Mailed check,20.35,33.20,None,...,None,None,None,None,Male,0,No,No,No,1
7041,11528,7208-PSIHR,2014-04-01,NaT,Two year,Yes,Bank transfer (automatic),104.30,7188.50,Fiber optic,...,Yes,Yes,Yes,Yes,Female,0,Yes,No,Yes,0


In [15]:
# Преобразуем список колонок в строку
columns_string = ", ".join(df.columns)

# Сохраняем строку в файл
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(columns_string)

In [24]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

		# обновите словарь stats
    stats.update(column_stat)


# Общая длина данных
stats["data_length"] = df.shape[0]

# Статистика по monthly_charges
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()

# Статистика по total_charges
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()

# Уникальные идентификаторы
stats["unique_customers_number"] = df["customer_id"].nunique()

# Количество пустых значений в колонке end_date
stats["end_date_nan"] = df["end_date"].isna().sum()

In [27]:
stats

{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': 18.25,
 'monthly_charges_max': 118.75,
 'monthly_charges_mean': 64.76169246059918,
 'monthly_charges_median': 70